In [26]:
from py_ecc.bn128 import G1, G2, multiply, add, curve_order, eq, Z1, pairing
import numpy as np
import galois

In [ ]:
GF = galois.GF(curve_order)
# takes about 2 minutes

In [1]:
x = GF(5)#random.randint(1,10)
y = GF(10)#random.randint(1,15)

v1 = x * x
v2 = v1 * x
v3 = y * y
v4 = v2 * y
out = v2 + GF(2) * v4 - GF(5) * x * v3 - GF(3) * y + GF(2)
print(1,out,x,y,v1,v2,v3,v4)

Lgf = GF(np.array([
    [0,0,1,0,0,0,0,0],
    [0,0,0,0,1,0,0,0],
    [0,0,0,1,0,0,0,0],
    [0,0,0,0,0,1,0,0],
    [0,0,curve_order - 5,0,0,0,0,0] # 113 - 5 = 108
    ]))

Rgf = GF(np.array([
    [0,0,1,0,0,0,0,0],
    [0,0,1,0,0,0,0,0],
    [0,0,0,1,0,0,0,0],
    [0,0,0,1,0,0,0,0],
    [0,0,0,0,0,0,1,0]
    ]))

Ogf = GF(np.array([
    [0,0,0,0,1,0,0,0],
    [0,0,0,0,0,1,0,0],
    [0,0,0,0,0,0,1,0],
    [0,0,0,0,0,0,0,1],
    [curve_order-2,1,0,3,0,curve_order-1,0,curve_order-2] # 113 - 2 = 111
    ]))

wgf = GF(np.array([GF(1),out,x,y,v1,v2,v3,v4]))

# assert all(np.multiply(np.dot(L,w), np.dot(R,w)) == np.dot(O,w)), "result contains an inequality"
result = Ogf.dot(wgf) == np.multiply(Lgf.dot(wgf),Rgf.dot(wgf))
assert result.all(), "result contains an inequality"


1 97 5 10 25 125 100 1250


In [2]:
target_poly = GF(np.array([1,2,3,4,5]))
# for each column in the matrix, compute the lagrange polynomial, resulting in a vector of polynomials
def interpolate_columns(column):
    return galois.lagrange_poly(target_poly,column)

L_vector_of_poly = np.apply_along_axis(interpolate_columns, 0, Lgf)
R_vector_of_poly = np.apply_along_axis(interpolate_columns, 0, Rgf)
O_vector_of_poly = np.apply_along_axis(interpolate_columns, 0, Ogf)
print(L_vector_of_poly)

[Poly(0, GF(21888242871839275222246405745257275088548364400416034343698204186575808495617))
 Poly(0, GF(21888242871839275222246405745257275088548364400416034343698204186575808495617))
 Poly(3648040478639879203707734290876212514758060733402672390616367364429301415936x^4 + 10944121435919637611123202872628637544274182200208017171849102093287904247810x^3 + 7296080957279758407415468581752425029516121466805344781232734728858602831868x^2 + 4x, GF(21888242871839275222246405745257275088548364400416034343698204186575808495617))
 Poly(16416182153879456416684804308942956316411273300312025757773653139931856371713x^4 + 21888242871839275222246405745257275088548364400416034343698204186575808495614x^3 + 16416182153879456416684804308942956316411273300312025757773653139931856371725x^2 + 10944121435919637611123202872628637544274182200208017171849102093287904247789x + 10, GF(21888242871839275222246405745257275088548364400416034343698204186575808495617))
 Poly(36480404786398792037077342908762125147580607334

In [12]:
def reduce_polynomials_to_1(vector_of_poly, witness):
    sum = GF(0)
    for i in range(len(vector_of_poly)):
        sum += vector_of_poly[i]*witness[i]
    return sum

Ux = reduce_polynomials_to_1(L_vector_of_poly,wgf)
Vx = reduce_polynomials_to_1(R_vector_of_poly,wgf)
Wx = reduce_polynomials_to_1(O_vector_of_poly,wgf)
print(Ux)
print(Vx)
print(Wx)
print(np.flip(np.polynomial.polynomial.polyfromroots([1,2,3,4])))
print(galois.Poly(np.flip(np.polynomial.polynomial.polyfromroots([1,2,3,4])),  field=GF))


7296080957279758407415468581752425029516121466805344781232734728858602831849x^4 + 3648040478639879203707734290876212514758060733402672390616367364429301416197x^3 + 3648040478639879203707734290876212514758060733402672390616367364429301414937x^2 + 7296080957279758407415468581752425029516121466805344781232734728858602833414x + 21888242871839275222246405745257275088548364400416034343698204186575808494842
13680151794899547013904003590785796930342727750260021464811377616609880309765x^4 + 9120101196599698009269335727190531286895151833506680976540918411073253539795x^3 + 8208091076939728208342402154471478158205636650156012878886826569965928186022x^2 + 12768141675239577212977070018066743801653212566909353367157285775502554955532x + 125
6384070837619788606488535009033371900826606283454676683578642887751277477581x^4 + 9120101196599698009269335727190531286895151833506680976540918411073253543130x^3 + 15504172034219486615757870736223903187721758116961357660119561298824531005611x^2 + 12768141675239577

In [16]:
UxVx = Ux*Vx
Tx = galois.Poly(np.flip(np.polynomial.polynomial.polyfromroots([1,2,3,4])).astype(int),  field=GF)
Hx = ((Ux*Vx) - Wx) // Tx

# Check Tx and Hx balances the polynomial correctly
lhs = Ux*Vx
rhs = Hx*Tx + Wx
assert lhs == rhs, "result is not balanced across LHS and RHS"

print(lhs)
print(rhs)

1824020239319939601853867145438106257379030366701336195308183682214650707866x^8 + 2888032045589904369601956313610334907516798080610448975904624163506530289817x^7 + 2584028672369914435959645122703983864620293019493559610019926883137421816206x^6 + 20216224319129330587213694195272344352617586564273142831332369144545712114344x^5 + 2584028672369914435959645122703983864620293019493559610019926883137421530681x^4 + 18848209139639375885823293836193764659583313789247140684851231382884724549857x^3 + 3952043851859869137350045481782563557654565794519561756501064644798409236943x^2 + 12768141675239577212977070018066743801653212566909353367157285775502555338037x + 21888242871839275222246405745257275088548364400416034343698204186575808398742
1824020239319939601853867145438106257379030366701336195308183682214650707866x^8 + 2888032045589904369601956313610334907516798080610448975904624163506530289817x^7 + 2584028672369914435959645122703983864620293019493559610019926883137421816206x^6 + 2021622431912933058

In [34]:
from functools import reduce
import random

def inner_product(ec_points, coeffs):
    return reduce(add, (multiply(point, int(coeff)) for point, coeff in zip(ec_points, coeffs)), Z1)

def generate_powers_of_tau_G1(tau, degree):
    return [multiply(G1, int(tau ** i)) for i in range(degree + 1)]

def generate_powers_of_tau_G2(tau, degree):
    return [multiply(G2, int(tau ** i)) for i in range(degree + 1)]

# tau = GF(8)
tau = GF(random.randint(1,curve_order-1))

# evaluate then convert
powers_of_tau_G1 = generate_powers_of_tau_G1(tau, Ux.degree)
powers_of_tau_G2 = generate_powers_of_tau_G2(tau, Ux.degree)

A_1 = inner_product(powers_of_tau_G1, Ux.coeffs)
B_2 = inner_product(powers_of_tau_G2, Vx.coeffs)
C_1prime = inner_product(powers_of_tau_G1, Wx.coeffs)
HxTx = Hx * Tx
HT_1 = inner_product(powers_of_tau_G1, HxTx.coeffs)
C = add(C_1prime, HT_1)

print("A_1",A_1)
print("B_2",B_2)
print("C_1prime",C_1prime)
print("HT_1",HT_1)
print("C",C)

eq(pairing(B_2, A_1), pairing(G2, C)); print("success pairing")


A_1 (19586754389903197392321542838362651396039431903866319760944757774120736779904, 15212093956528139374355432165904971151761476114945075994988747668270913101928)
B_2 ((1679319053306018581552402856160648024692192561473792684496160399933544664611, 19869724381476644271600942059125004140054085355619517020880676158218617502785), (6101697475570065326171373198929280727685710408888290846514975319540852942313, 1375845418767308100336681377736573777560298382943267776252525462041439512043))
C_1prime (15000478207967489890828385916500178681809437502225540019072804606978863114132, 20384112167488133494329829305342784001708637191639680820412813060322610527724)
HT_1 (9273704774377297886789976705710823684688032310981707918662713613614473419613, 4469481318782167812599401290984763027314418055141929360277924556185850154465)
C (11051135031331623782448847978485506765514919513922776832572394550073955066014, 18766883177904295314440207078800230744579170597602731021393558669997530994243)
success pairing
